# Astropy Units, Quantities, and Constants

Astropy includes a powerful framework for units that allows users to attach units to scalars and arrays.  These quantities can be manipulated or combined, while keeping track of the units.

For more information about the features presented below, please see the
[astropy.units](https://docs.astropy.org/en/stable/units/index.html) docs.

Also note that this tutorial assumes you have little or no knowledge of the astropy units docs.  If you're moderately familiar with them and are interested in some more complex examples, you might instead prefer the Astropy tutorial on ["Using Astropy Quantities for astrophysical calculations"](http://learn.astropy.org/Quantities.html).

Table of contents:

* <a href="#Representing-units">Representing units</a>
* <a href="#Composite-units">Composite units</a>
* <a href="#Quantity-objects">Quantity objects</a>
* <a href="#Quantity-attributes">Quantity attributes</a>
* <a href="#Quantity-arithmetic-operations">Quantity arithmetic operations</a>
* <a href="#Combining-Quantities">Combining Quantities</a>
* <a href="#Converting-units">Converting units</a>
* <a href="#Decomposing-units">Decomposing units</a>
* <a href="#Integration-with-Numpy-functions">Integration with NumPy functions</a>
* <a href="#Defining-new-units">Defining new units</a>
* <a href="#Using-physical-constants">Using physical constants</a>
* <a href="#Equivalencies">Equivalencies</a>
* <a href="#Performance-consideration">Performance consideration</a>
* <a href="#Putting-it-all-together:--a-concise-example">Putting it all together: a concise example</a>
* <a href="#Exercises">Exercises</a>

## Representing units

First, we need to import the astropy units subpackage (**`astropy.units`**).  Because we probably want to use units in many expressions, it is most concise to rename the subpackage as **`u`**.  This is the standard convention, but note that this will conflict with any variable called **`u`**:

In [ ]:
import astropy.units as u

In [ ]:
# We will also import numpy here, which we use later.
# "np" is the common naming standard for this import.
import numpy as np

Units can then be accessed as **`u.<unit>`**.  For example, the meter unit is:

In [ ]:
u.m

Units have docstrings, which give some explanatory text about them:

In [ ]:
u.m.__doc__

In [ ]:
u.pc.__doc__

and a physical type:

In [ ]:
u.m.physical_type

In [ ]:
u.s.physical_type

Many units also have aliases:

In [ ]:
u.m.aliases

In [ ]:
u.meter

In [ ]:
u.arcsec.aliases

In [ ]:
u.arcsecond

SI and cgs units are available by default, but Imperial units require the **`imperial`** prefix:

In [ ]:
# This is undefined.
u.inch

In [ ]:
# Use this instead.
u.imperial.inch

Please see the complete list of [available units](https://astropy.readthedocs.org/en/stable/units/index.html#module-astropy.units.si).

## Composite units

Composite units are created using Python numeric operators (e.g., "`*`" (multiplication), "`/`" (division), and "`**`" (power)).

In [ ]:
u.km / u.s

In [ ]:
u.imperial.mile / u.h

In [ ]:
(u.eV * u.Mpc) / u.Gyr

In [ ]:
u.cm**3

In [ ]:
u.m / u.kg / u.s**2

## ``Quantity`` objects
The most useful feature of units is the ability to attach them to scalars or arrays, creating `Quantity` objects. A `Quantity` object contains both a value and a unit.  The most convenient way to create a `Quantity` object is by multiplying the value with its unit.

In [ ]:
3.7 * u.au  # Quantity object

An equivalent (but more verbose) way of doing the same thing is to use the `Quantity` object's initializer, as demonstrated below.  In general, the more concise form (above) is preferred, as it is closer to how such a quantity would actually be written in text.  The initalizer form has more options, though, which you can learn about from the [Astropy reference documentation on Quantity](https://docs.astropy.org/en/stable/api/astropy.units.quantity.Quantity.html).

In [ ]:
u.Quantity(3.7, unit=u.au)

In [ ]:
u.Quantity(1 * u.cm, unit=u.m)

Where quantities really shine is when you make an array `Quantity` object.

In [ ]:
# We want to create the composite unit first, hence the parenthesis.
x = [1.2, 6.8, 3.7] * (u.pc / u.year)
x

## `Quantity` attributes

The units and value of a `Quantity` can be accessed separately via the ``value`` and ``unit`` attributes:

In [ ]:
q = 5. * u.Mpc
q

In [ ]:
q.value

In [ ]:
q.unit

In [ ]:
x = [1.2, 6.8, 3.7] * (u.pc / u.year)
x

In [ ]:
x.value

In [ ]:
x.unit

## `Quantity` arithmetic operations

"`*`" (multiplication), "`/`" (division), and "`**`" (power) operations can be performed on `Quantity` objects with `float`/`int` values.

In [ ]:
q = 3.1 * u.km

In [ ]:
q * 2

In [ ]:
q / 2.

In [ ]:
1 / q

In [ ]:
q ** 2

## Combining Quantities

Quantities can be combined using Python numeric operators:

In [ ]:
q1 = 3. * (u.m / u.s)
q1

In [ ]:
q2 = 5. * (u.cm / u.s / u.g**2)
q2

In [ ]:
q1 * q2

In [ ]:
q1 / q2  # Note the "second" unit cancelled out.

In [ ]:
# Sometimes, one more step needed for "clean" output unit.
# Also see: Decomposing units (below)
(q1 / q2).to(u.g ** 2)

In [ ]:
q1 ** 2

In [ ]:
x = [1.2, 6.8, 3.7] * (u.pc / u.year)
x * 3  # Element-wise multiplication.

When adding or subtracting quantities, the units must be **compatible** (but not necessarily identical).

In [ ]:
# Add two quantities.
(3 * u.m) + (5 * u.m)

Here we add two distance quantities that do not have identical units:

In [ ]:
(3 * u.km) + (5 * u.cm)

In [ ]:
# This will fail because the units are incompatible.
(3 * u.km) + (5. * u.km / u.s)

## Converting units

Units can be converted to other equivalent units.

In [ ]:
q = 2.5 * u.year
q

In [ ]:
# Convert year to seconds.
q.to(u.s)

In [ ]:
# Convert degree squared to steradian.
(7. * u.deg**2).to(u.sr)

In [ ]:
# Convert mph to kph.
(55. * (u.imperial.mile / u.h)).to(u.km / u.h)

**Important Note**:  Converting a unit (not a `Quantity`) gives only the scale factor:

In [ ]:
u.Msun.to(u.kg)

To keep the units, use a `Quantity` (value and unit) object:

In [ ]:
(1. * u.Msun).to(u.kg)

## Decomposing units

The units of a `Quantity` object can be decomposed into a set of base units using the
``decompose()`` method. By default, units will be decomposed to SI unit bases:

In [ ]:
q = 8. * (u.cm * u.pc / u.g / u.year**2)
q

In [ ]:
q.decompose()

To decompose into cgs unit bases:

In [ ]:
q.decompose(u.cgs.bases)

In [ ]:
u.cgs.bases

In [ ]:
u.si.bases

Units will not cancel out unless they are identical:

In [ ]:
q = 7 * u.m / (7 * u.km)
q

But they will cancel by using the `decompose()` method:

In [ ]:
x = q.decompose()
x  # This is a "dimensionless" Quantity.

In [ ]:
repr(x.unit)

## Integration with NumPy functions

Most [NumPy](https://www.numpy.org) functions understand `Quantity` objects:

In [ ]:
np.sin(30)  # np.sin assumes the input is in radians.

In [ ]:
np.sin(30 * u.degree)  # Awesome!

In [ ]:
q = 100 * (u.kg * u.kg)
np.sqrt(q)

In [ ]:
x = np.arange(10) * u.km
x

In [ ]:
np.mean(x)

Some NumPy "ufuncs" (universal functions) require dimensionless quantities.

In [ ]:
np.log10(4 * u.m)  # This is not possible.

In [ ]:
np.log10(4 * u.m / (4 * u.km))  # Note that the units cancelled.

Care needs to be taken with dimensionless units.

For example, passing ordinary values to an inverse trigonometric function gives a result without units:

In [ ]:
np.arcsin(1.0)

`u.dimensionless_unscaled` creates a ``Quantity`` with a "dimensionless unit" and therefore gives a result *with* units:

In [ ]:
np.arcsin(1.0 * u.dimensionless_unscaled)

In [ ]:
np.arcsin(1.0 * u.dimensionless_unscaled).to(u.degree)

**Important Note:**  In-place array operations do not work with units.

In [ ]:
a = np.arange(10.)
a *= 1.0 * u.kg  # In-place operator will fail.

Assign to a *new* array instead:

In [ ]:
a = a * 1.0 * u.kg
a

Also, Quantities lose their units with some NumPy operations, for example:

* np.append
* np.dot
* np.hstack
* np.vstack
* np.where
* np.choose
* np.vectorize

See [Quantity Known Issues](https://docs.astropy.org/en/stable/known_issues.html#quantities-lose-their-units-with-some-operations) for more details.

## Defining new units

You can also define custom units for something that isn't built in to Astropy.

Let's define a unit called **"sol"** that represents a Martian day.

In [ ]:
sol = u.def_unit('sol', 1.0274912510 * u.day)

In [ ]:
(1. * u.yr).to(sol)  # 1 Earth year in Martian sol units.

Now let's define Mark Watney's favorite unit, the [**Pirate-Ninja**](https://en.wikipedia.org/wiki/List_of_humorous_units_of_measurement#Pirate_Ninja):

In [ ]:
pirate_ninja = u.def_unit('☠️👤', 1.0 * (u.kW * u.hr / sol))

In [ ]:
5.2 * pirate_ninja

In [ ]:
# Mars oxygenator power requirement for 6 people.
(44.1 * pirate_ninja).to(u.W)

## Using physical constants

The [astropy.constants](https://docs.astropy.org/en/stable/constants/index.html) module contains physical constants relevant for astronomy.  They are defined as ``Quantity`` objects using the ``astropy.units`` framework.

Please see the complete list of [available physical constants](https://docs.astropy.org/en/stable/constants/index.html#module-astropy.constants).  Additions are welcome!

In [ ]:
from astropy import constants as const

In [ ]:
const.G

In [ ]:
const.c

In [ ]:
const.L_sun

Constants are Quantities, thus they can be coverted to other units:

In [ ]:
const.L_sun.to(u.erg / u.s)

Also note that even constants are not constant. `astropy.constants` supports [different versions](https://docs.astropy.org/en/stable/constants/index.html#collections-of-constants-and-prior-versions).

In [ ]:
import astropy

In [ ]:
astropy.physical_constants.get()

In [ ]:
astropy.astronomical_constants.get()

In [ ]:
from astropy.constants import iau2012
iau2012.L_sun

In [ ]:
# Changing the entire science state like this after
# imports is currently not possible, see
# https://github.com/astropy/astropy/issues/8781
astropy.astronomical_constants.set('iau2012')

## Equivalencies

Equivalencies can be used to convert quantities that are not strictly the same physical type, but in a specific context are interchangable.  A familiar physics example is the mass-energy equivalency: these are strictly different physical types, but it is often understood that you can convert between the two using $E=mc^2$:

In [ ]:
from astropy.constants import m_p  # Proton mass

In [ ]:
# This raises an error because mass and energy are different units.
(m_p).to(u.eV)

In [ ]:
# This succeeds using equivalencies.
(m_p).to(u.MeV, u.mass_energy())

This concept extends further in `astropy.units` to include some common practical astronomy situations where the units have no direct physical connection, but it is often useful to have a "quick shorthand."  For example, astronomical spectra are often given as a function of wavelength, frequency, or even energy of the photon.  Suppose you want to find the [Lyman-limit](https://en.wikipedia.org/wiki/Lyman_limit) wavelength:

In [ ]:
# This raises an error.
(13.6 * u.eV).to(u.Angstrom)

Normally, you can convert `u.eV` only to the following units:

In [ ]:
u.eV.find_equivalent_units()

But by using a spectral equivalency, you can also convert `u.eV` to the following units:

In [ ]:
u.eV.find_equivalent_units(equivalencies=u.spectral())

In [ ]:
# Now back to Lyman-limit.
(13.6 * u.eV).to(u.Angstrom, equivalencies=u.spectral())

Or if you remember the 21cm HI line, but cannot remember the frequency, you could do:

In [ ]:
(21. * u.cm).to(u.GHz, equivalencies=u.spectral())

To go one step further, the units of a spectrum's *flux* are further complicated by being dependent on the units of the spectrum's "X-axis" (i.e., $f_{\lambda}$ for flux per unit wavelength or $f_{\nu}$ for flux per unit frequency).  `astropy.units` supports this use case, but it is necessary to supply the location in the spectrum where the conversion is done:

In [ ]:
q = 1e-18 * (u.erg / u.s / u.cm**2 / u.AA)
q

In [ ]:
q.to(u.uJy, equivalencies=u.spectral_density(1. * u.um))

There's a lot of flexibility with equivalencies, including a variety of other useful built-in equivalencies.  So if you want to know more, you might want to check out the [equivalencies narrative documentation](https://docs.astropy.org/en/stable/units/equivalencies.html) or the [astropy.units.equivalencies reference docs](https://docs.astropy.org/en/stable/units/index.html#module-astropy.units.equivalencies).

## Domain-specific units

Some astropy modules define more specific units and equivalencies.

For example, ``astropy.cosmology`` defines units for cosmological redshift and
the Hubble parameter and equivalencies for both.

In [ ]:
import astropy.cosmology.units as cu

cu.__all__

In [ ]:
z = 1100 * cu.redshift
repr(z)

Redshift isn't a "real" unit (so the ``dimensionless_redshift`` equivalency
is turned on by default) but it is very useful for converting between cosmological
distance measures, for example the background temperature. We can do this
in a specific cosmological context:

In [ ]:
from astropy.cosmology import Planck18  # cosmological context

z.to(u.K, equivalencies=cu.redshift_temperature(Planck18))

If you want to know more about cosmology and units, you might want to check out the [astropy.cosmology.units  reference docs](https://docs.astropy.org/en/stable/cosmology/units.html#cosmological-units-and-equivalencies).

# Performance consideration

For more details about performance on big datasets, see [units performance tips](https://docs.astropy.org/en/stable/units/index.html#performance-tips).

When working with a big data array, the `<<` operator can be used to speed up Quantity initialization. Let's consider this array:

In [ ]:
big_array = np.random.random(10_000_000)

In [ ]:
# Defining the Quantity the simplest way without parenthesis.
%timeit big_array * u.m / u.s / u.kg / u.sr

In [ ]:
# Defining the Quantity with the * operator and parenthesis.
%timeit big_array * (u.m / u.s / u.kg / u.sr)

In [ ]:
# Defining the Quantity with the << operator (parenthesis not needed).
%timeit big_array << u.m / u.s / u.kg / u.sr

Replace the ``Quantity`` values below with your slow/fast timing to see how much things sped up on your machine. For example:

In [ ]:
((89.4 * u.ms) / (67.1 * u.us)).to(u.dimensionless_unscaled)

# Putting it all together:  a concise example

Let's estimate the (circular) orbital speed of the Earth around the Sun using Kepler's Law:

$$v = \sqrt{\frac{G M_{\odot}}{r}}$$

In [ ]:
v = np.sqrt(const.G * 1 * u.M_sun / (1 * u.au))
v

That's a velocity unit... but it sure isn't obvious when you look at it!

Let's use a variety of the available ``Quantity`` methods to get something more sensible:

In [ ]:
v.decompose()  # Remember that the default uses SI bases

In [ ]:
v.decompose(u.cgs.bases)

In [ ]:
v.to(u.km / u.s)

# Exercises

## Exercise 1

The *James Webb Space Telescope (JWST)* is in a halo orbit around the second Sun-Earth Lagrange (L2) point:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;☀️ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 🌎 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; L2 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *(not to scale)*


L2 is located at a distance from the Earth (opposite the Sun) of approximately:

$$ r \approx R \left(\frac{M_{earth}}{3 M_{sun}}\right) ^{(1/3)} $$

where $R$ is the Sun-Earth distance.

Calculate the Earth-L2 distance in kilometers and miles.

*Hints*:

* $M_{earth}$ and $M_{sun}$ are defined [constants](https://docs.astropy.org/en/stable/constants/#reference-api) 

* The mile unit is defined as ``u.imperial.mile`` (see [imperial units](https://docs.astropy.org/en/v0.2.1/units/index.html#module-astropy.units.imperial))

In [ ]:
# Answer here (km)

In [ ]:
# Answer here (mile)

## Exercise 2

The L2 point is about 1.5 million kilometers away from the Earth opposite the Sun.
The total mass of the *James Webb Space Telescope (JWST)* is about 6500 kg.

Using the value you obtained above for the Earth-L2 distance, calculate the gravitational force in Newtons between: 

* *JWST* (at L2) and the Earth
* *JWST* (at L2) and the Sun

*Hint*: the gravitational force between two masses separated by a distance *r* is:

$$ F_g = \frac{G m_1 m_2}{r^2} $$

In [ ]:
# Answer here (JWST and Earth)

In [ ]:
# Answer here (JWST and Sun)

## Exercise 3

Calculate the [Schwarzschild radius](https://en.wikipedia.org/wiki/Schwarzschild_radius) in units of solar radii of the Sgr A*, the Milky Way's supermassive black hole with $M = 4.31 \times 10^6 M_\odot$, given

$$r_\mathrm{s} = \frac{2 G M}{c^2}$$

Also calculate the angular size of the event horizon on the sky in microarcseconds, given the distance to the galactic center $d_{center} = 7.94$ kpc, given

$$\theta = \mathrm{arctan}\frac{2 r_\mathrm{s}}{d_{center}}$$

In [ ]:
# Answer radius here

In [ ]:
# Answer angular size here

## Exercise 4

We can make a very rough estimate of the temperature of material in the vicinity of Sgr A* by assuming hydrostatic equilibrium, so that the thermal energy of the gas balances the gravitational force:

$$ kT \sim GM m_p /R $$

where $m_p$ is the mass of a proton and $R$ is the distance away from the black hole. Using Astropy constants and the properties of Sgr A* described in Example 3, compute the temperature of the gas required to balance the black hole's gravitation pull at 1 million Schwarzschild radii derived above. Use this equation:

$$ T = \frac{G M m_p}{10^6 r_s k} $$

Then use the [Astropy units temperature equivalencies](https://docs.astropy.org/en/stable/units/equivalencies.html#temperature-equivalency) to find the energy of that gas.

In [ ]:
# Answer temperature here

In [ ]:
# Answer energy here